In [1]:
!pip install --upgrade pip setuptools
!pip install pandas
!pip install numpy
!pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np

In [10]:


# Tiếp tục với hàm recommend_products
def cosine_similarity(user1_ratings, user2_ratings):
    # Tính dot product
    dot_product = np.dot(user1_ratings, user2_ratings)
    
    # Tính độ dài của vectors
    norm_user1 = np.linalg.norm(user1_ratings)
    norm_user2 = np.linalg.norm(user2_ratings)
    
    # Tránh chia cho 0
    if norm_user1 == 0 or norm_user2 == 0:
        return 0
    
    # Tính cosine similarity
    similarity = dot_product / (norm_user1 * norm_user2)
    
    return similarity

def recommend_products(user_id, prod_id):
    # Lấy ratings của user hiện tại
    user_ratings = rating_matrix.loc[user_id]
    
    # Tìm tất cả các user khác có đánh giá cho sản phẩm đang xem
    users_with_rating = rating_matrix[rating_matrix[prod_id] > 0]
    
    # Tính độ tương đồng cosine giữa user hiện tại và các user có đánh giá cho sản phẩm đang xem
    similarities = users_with_rating.apply(lambda x: cosine_similarity(user_ratings, x), axis=1)
    
    # Sắp xếp và lấy k người dùng có độ tương đồng cao nhất
    similar_users = similarities.nlargest(k+1).iloc[1:]  # bỏ đi user hiện tại
    
    # Tạo danh sách sản phẩm gợi ý từ các người dùng tương đồng
    recommended_products = set()
    for user, similarity in similar_users.items():
        # Lấy ratings của người dùng tương đồng
        neighbor_ratings = rating_matrix.loc[user]
        # Lấy các sản phẩm được đánh giá cao hơn 3
        top_rated_products = neighbor_ratings[neighbor_ratings > 3].index.tolist()
        # Thêm vào danh sách sản phẩm gợi ý
        recommended_products.update(top_rated_products)
    
    # Loại bỏ sản phẩm đã được người dùng hiện tại đánh giá
    recommended_products.discard(prod_id)
    
    return recommended_products

In [14]:
df = pd.read_csv('data/ratings.csv', header=None)
df.columns = ['user', 'product', 'rating', 'timestamp']
df.drop(['timestamp'], axis=1, inplace=True)
# Xây dựng ma trận đánh giá# Xử lý các giá trị trùng lặp bằng cách chọn giá trị trung bình
rating_matrix = df.pivot_table(index='user', columns='product', values='rating', aggfunc='mean')
rating_matrix = rating_matrix.fillna(0)  # Chuyển các giá trị NaN thành 0

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
user_id = 6
prod_id = 2
k = 5  # số lượng người dùng tương đồng được chọn
recommended_products = recommend_products(user_id, prod_id)
print("Các sản phẩm được gợi ý cho user {} khi xem sản phẩm {}: {}".format(user_id, prod_id, recommended_products))